<a href="https://colab.research.google.com/github/pounda/soccerdetectron/blob/master/Soccer_Detectron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Soccer Detectron Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">




# Install detectron2

In [1]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
     |████████████████████████████████| 276kB 8.6MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=fe832bc766644fb3f50bf6bb4dd591997bca1bd78c5709d3e72cbfb7f53a4914
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-9mkqoxz0
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-9mkqoxz0
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-lin

In [2]:
# install detectron2:
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
     |████████████████████████████████| 6.2MB 703kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1.post200513-cp36-none-any.whl size=40893 sha256=9c77b394b55288686e706edeb2c03351db84a691e30555cf4f0da1500112a2ff
  Stored in directory: /root/.cache/pip/wheels/a2/f6/77/551770c4b8cd75e9335cd0acf59c08d60a8684048b19da6702
Successfully built fvcore


# Install common libraries and utilities


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


In [0]:
#!gdown https://drive.google.com/open?id=1CPhAlWM2DUSl4fI51jaxGYyjKwmyYpoC
!gdown https://drive.google.com/uc?id=1CPhAlWM2DUSl4fI51jaxGYyjKwmyYpoC
!ls

!unzip -q /content/soccer.zip
!rm /content/soccer.zip
!mkdir '/content/output/'
#!mv soccer_zip/ output/soccer.zip

Downloading...
From: https://drive.google.com/uc?id=1CPhAlWM2DUSl4fI51jaxGYyjKwmyYpoC
To: /content/soccer.zip
1.60GB [00:18, 87.1MB/s]


In [0]:
!ls

# Run a pre-trained detectron2 model

**List folders**

In [0]:
!ls /content/
#!ls /content/sample_data
!ls /content/soccer/



```
# This is formatted as code
```

# **Load a soccer image**


In [0]:
im = cv2.imread("/content/soccer/img1202.png")
cv2_imshow(im)

# Create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [0]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

# Look at the outputs

In [0]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
outputs["instances"].pred_classes
outputs["instances"].pred_boxes

# Use `Visualizer` to draw the predictions on the image.

In [0]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

# Create masks, classes and boxes and save output

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from detectron2.structures import BitMasks, Boxes, BoxMode, PolygonMasks

masks = np.asarray(outputs["instances"].pred_masks.cpu())
classes = outputs["instances"].pred_classes.cpu()
boxes = outputs["instances"].pred_boxes.to(torch.device("cpu"))
count = 0
for m in masks:
 b=boxes[count].tensor.numpy()[0]
 print(b)
 x = b[2]
 print(x)
 xi = int(x)
 print(xi)
 print("class:")
 print(classes[2].numpy())
 print("after")
 if isinstance(m, dict):
  print("dict")
 if isinstance(m, list):
  print("list")
 if isinstance(m, np.ndarray):
  print("ndarray")
  print(m.dtype)
  assert m.shape[1] != 2, m.shape
  print(m.shape)
  m=m*1
  _mask = m.astype(np.uint8)
  print(_mask[0][0])
  plt.imshow(_mask)
  plt.show()
  print(_mask.dtype)
  print(_mask)
  print(_mask.shape)
  img = v.get_image()
  print(img.shape)
  im_alpha = cv2.cvtColor(im,cv2.COLOR_BGR2BGRA)
  new_img = cv2.bitwise_and(im_alpha, im_alpha, mask=_mask)
  print(new_img.shape)
  color = ('b','g','r')

  cv2.imwrite('/content/output/new_img' + str(count) + '.png',new_img)
  crop_img = new_img[int(b[1]):int(b[3]), int(b[0]):int(b[2])]

  if not  cv2.imwrite('/content/output/crop_img_tp' + str(count) + '.png',crop_img):
     raise Exception("Could not write image")
  plt.imshow(crop_img)
  plt.show()
  count += 1

# Dominant Colors class

In [0]:
class DominantColors:

    CLUSTERS = None
    IMAGE = None
    FLAT_IMAGE = None
    COLORS = None
    LABELS = None
   
    def __init__(self, image, clusters=3):
        self.CLUSTERS = clusters
        self.IMAGE = image
   
    def dominantColors(self):

        img = self.IMAGE
               
        #reshaping to a list of pixels
        img = img.reshape((img.shape[0] * img.shape[1], 4))
       
        #save image after operations
        self.FLAT_IMAGE = img
       
        #using k-means to cluster pixels
        kmeans = KMeans(n_clusters = self.CLUSTERS)
        kmeans.fit(img)
       
        #getting the colors as per dominance order
        self.COLORS = kmeans.cluster_centers_
       
        #save labels
        self.LABELS = kmeans.labels_
       
        return self.COLORS.astype(int)
               
    def plotHistogram(self):
       
        #labels form 0 to no. of clusters
        numLabels = np.arange(0, self.CLUSTERS+1)
       
        #create frequency count tables    
        (hist, _) = np.histogram(self.LABELS, bins = numLabels)
        hist = hist.astype("float")
        hist /= hist.sum()
       
        #appending frequencies to cluster centers
        colors = self.COLORS
       
        #descending order sorting as per frequency count
        colors = colors[(-hist).argsort()]
        hist = hist[(-hist).argsort()]
       
        #creating empty chart
        chart = np.zeros((50, 500, 3), np.uint8)
        start = 0
       
        #creating color rectangles
        for i in range(self.CLUSTERS):
          if colors[i][3] > 1:
            end = start + hist[i] * 500
           
            #getting rgb values
            r = colors[i][0]
            g = colors[i][1]
            b = colors[i][2]
           
            #using cv2.rectangle to plot colors
            cv2.rectangle(chart, (int(start), 0), (int(end), 50), (r,g,b), -1)
            start = end
       
        #display chart
        plt.figure()
        plt.axis("off")
        plt.imshow(chart)
        plt.show()
       
    def rgb_to_hex(self, rgb):
        return '#%02x%02x%02x' % (int(rgb[0]), int(rgb[1]), int(rgb[2]))
   
    def plotClusters(self):
        #plotting
        fig = plt.figure()
        ax = Axes3D(fig)        
        for label, pix in zip(self.LABELS, self.FLAT_IMAGE):
            ax.scatter(pix[0], pix[1], pix[2], color = self.rgb_to_hex(self.COLORS[label]))
        plt.show()
       
    def colorPixels(self):
       
        shape = self.IMAGE.shape
       
        img = np.zeros((shape[0] * shape[1], 4))
        labels = self.LABELS

        for i,color in enumerate(self.COLORS):
           
            indices = np.where(labels==i)[0]
           
            for index in indices:
                img[index] = color
       
        img = img.reshape((shape[0], shape[1], 4)).astype(int)
       
        #display img
        plt.figure()
        plt.axis("off")
        plt.imshow(img)
        plt.show()


import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

from numpy.testing import rundocs

for i in range(13):
    print(i)
    img = ('/content/output/crop_img_tp' + str(i) + '.png')
    img = cv2.imread(img, cv2.IMREAD_UNCHANGED)

    #convert to RGB from BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)

    #no. of clusters
    clusters = 5

    #initialize using constructor
    dc = DominantColors(img, clusters)

    #print dominant colors
    colors = dc.dominantColors()
    print(colors)

    #display clustered points
    dc.plotClusters()

    #display dominance order
    colors = dc.plotHistogram()
    print(colors)

# Check output directory

In [0]:
!ls output